<a href="https://colab.research.google.com/github/cytric-74/gta6-sales-prediction-/blob/main/gta6_sales_prediction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install vaderSentiment

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 126.0/126.0 kB 3.0 MB/s eta 0:00:00


In [ ]:
import os
import requests
import pandas as pd
from bs4 import BeautifulSoup
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer
from googleapiclient.discovery import build

In [ ]:
!mkdir data


# ***Twitter data collection***

In [ ]:
TOKKEN_TWIITER = "we need a token here"

In [ ]:
def fetch_twitter_data(query, max_results=50):
    headers = {"Authorization": f"Bearer {TOKKEN_TWIITER}"}
    url = f"https://api.twitter.com/2/tweets/search/recent?query={query}&tweet.fields=created_at,public_metrics&max_results={max_results}"
    response = requests.get(url, headers=headers)
    tweets = response.json()('data', [])

    df = pd.DataFrame([{
        'text': tweet['text'],
        'created_at': tweet['created_at'],
        'likes': tweet['public_metrics']['like_count']
    } for tweet in tweets])

    df.to_csv('data/twitter_data.csv', index=False)
    return df

# ***Youtube data collection***

In [ ]:
from googleapiclient.discovery import build
import pandas as pd

In [ ]:
API_KEY = 'need to get a youtube api key asap'
youtube = build('youtube', 'v3', developerKey=API_KEY)

In [ ]:
def fetch_youtube_comments(query):
    search_response = youtube.search().list(
        q=query, part='snippet', type='video', maxResults=5
    ).execute()

    video_ids = [item['id']['videoId'] for item in search_response['items']]
    all_comments = []

    for vid in video_ids:
        comment_response = youtube.commentThreads().list(
            part='snippet', videoId=vid, textFormat='plainText', maxResults=50
        ).execute()

        for item in comment_response.get("items", []):
            comment = item['snippet']['topLevelComment']['snippet']['textDisplay']
            all_comments.append(comment)

    df = pd.DataFrame({'comment': all_comments})
    df.to_csv('data/youtube_data.csv', index=False)
    return df

# ***Web Scrapping***

---



In [ ]:
def scrape_news(query):
    formatted_query = query.replace(" ", "%20")
    search_url = f"https://news.google.com/search?q={formatted_query}"
    response = requests.get(search_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    articles = []
    for a in soup.select('article h3'):
        if a.a:
            title = a.text.strip()
            link = "https://news.google.com" + a.a['href'][1:]
            articles.append({'title': title, 'link': link})

    df = pd.DataFrame(articles)
    df.to_csv('data/news_articles.csv', index=False)
    return df